In [ ]:
!pip install sqlalchemy clickhouse-connect python-dotenv

In [3]:
load_dotenv(override=True)

NameError: name 'load_dotenv' is not defined

In [3]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables from .env file
load_dotenv(override=True)

def get_postgres_engine():
    try:
        engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}".format(
                                user=os.getenv('pg_user'),
                                password=os.getenv('pg_password'),
                                host=os.getenv('pg_host'), 
                                port=os.getenv('pg_port'), 
                                dbname=os.getenv('pg_dbname')
                                )
                              )
        with engine.connect() as connection:
            print("Connection to the PostgreSQL database was successful!")
    except Exception as e:
        print("Error: Unable to connect to the PostgreSQL database.")
        print(e)

if __name__ == "__main__":
    get_postgres_engine()


Error: Unable to connect to the PostgreSQL database.
(psycopg2.OperationalError) connection to server on socket "@localhost/.s.PGSQL.5432" failed: Invalid argument (0x00002726/10022)
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [30]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Define the function to get the Snowflake connection
def get_snowflake_connection():
    connection_string = (
        "snowflake://{user}:{password}@{account}/{database}/{schema}"
        "?warehouse={warehouse}&role={role}"
    )
    connection_url = connection_string.format(
        user=os.getenv('sn_user'),
        password=os.getenv('sn_password'),
        account=os.getenv('sn_account_identifier'),
        database=os.getenv('sn_database'),
        schema=os.getenv('sn_schema'),
        warehouse=os.getenv('sn_warehouse'),
        role=os.getenv('sn_role')
    )
    engine = create_engine(connection_url)
    return engine

# Define a function to test the connection
def test_snowflake_connection(engine):
    try:
        # Create a connection
        connection = engine.connect()
        # Run a simple query
        query = "SELECT current_timestamp()"
        result = connection.execute(query)
        # Fetch the result
        current_timestamp = result.fetchone()
        print("Connection successful! Current timestamp:", current_timestamp)
        connection.close()
    except Exception as e:
        print("Connection failed:", e)


In [19]:
import snowflake.connector
import os

def get_snowflake_connection():
    conn = snowflake.connector.connect(
        user=os.getenv('sn_user'),
        password=os.getenv('sn_password'),
        account=os.getenv('sn_account_identifier'),
        warehouse=os.getenv('sn_warehouse'),
        database=os.getenv('sn_database'),
        schema=os.getenv('sn_schema'),
        role=os.getenv('sn_role')
    )
    return conn

def test_snowflake_connection():
    try:
        conn = get_snowflake_connection()
        cur = conn.cursor()
        cur.execute("SELECT current_timestamp()")
        result = cur.fetchone()
        print("Connection successful! Current timestamp:", result)
        cur.close()
        conn.close()
    except Exception as e:
        print("Connection failed:", e)

if __name__ == "__main__":
    test_snowflake_connection()


Connection successful! Current timestamp: (datetime.datetime(2024, 7, 7, 16, 5, 25, 937000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),)


In [50]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import os
from snowflake.sqlalchemy import URL

def test_snowflake_connection():
    try:       
        # Create Snowflake URL
        snowflake_url = URL(
            user=os.getenv('sn_user'),
            password=os.getenv('sn_password'),
            account=os.getenv('sn_account_identifier'),
            database=os.getenv('sn_database'),
            schema=os.getenv('sn_schema'),
            warehouse=os.getenv('sn_warehouse'),
            role=os.getenv('sn_role')
        )

        # Create SQLAlchemy engine
        engine = create_engine(snowflake_url)

        # Test the connection by executing a simple query
        with engine.connect() as connection:
            result = connection.execute("SELECT CURRENT_DATE()").fetchone()
            print(f"Connected to Snowflake. Current date in Snowflake: {result[0]}")

    except SQLAlchemyError as e:
        print(f"Error connecting to Snowflake: {e}")

if __name__ == "__main__":
    test_snowflake_connection()


Connected to Snowflake. Current date in Snowflake: 2024-07-07


In [53]:
import pandas as pd
from sqlalchemy import create_engine
import os
from snowflake.sqlalchemy import URL

def create_snowflake_engine():
    try:
        # Retrieve Snowflake connection parameters from environment variables
        snowflake_user = os.getenv('sn_user')
        snowflake_password = os.getenv('sn_password')
        snowflake_account = os.getenv('sn_account_identifier')
        snowflake_warehouse = os.getenv('sn_warehouse')
        snowflake_database = os.getenv('sn_database')
        snowflake_schema = os.getenv('sn_schema')
        snowflake_role = os.getenv('sn_role')

        # Create Snowflake URL
        snowflake_url = URL(
            user=snowflake_user,
            password=snowflake_password,
            account=snowflake_account,
            database=snowflake_database,
            schema=snowflake_schema,
            warehouse=snowflake_warehouse,
            role=snowflake_role
        )

        # Create SQLAlchemy engine and return it
        engine = create_engine(snowflake_url)
        return engine

    except Exception as e:
        print(f"Error creating Snowflake engine: {e}")
        return None

def load_csv_to_snowflake(engine, schema, table_name, csv_file_path):
    try:
        # Load CSV into Pandas DataFrame
        df = pd.read_csv(csv_file_path)

        # Write DataFrame to Snowflake table using SQLAlchemy Engine
        df.to_sql(table_name, con=engine, schema=schema, if_exists='append', index=False)

        print(f"Successfully loaded CSV data into Snowflake table '{schema}.{table_name}'")

    except Exception as e:
        print(f"Error loading CSV data to Snowflake: {e}")

if __name__ == "__main__":
    # Example usage
    snowflake_engine = create_snowflake_engine()

    if snowflake_engine:
        schema = 'STG'
        table_name = 'tripsdata'
        csv_file_path = './raw_data/tripsdata.csv'

        load_csv_to_snowflake(snowflake_engine, schema, table_name, csv_file_path)
    else:
        print("Failed to create Snowflake engine.")


Error loading CSV data to Snowflake: 'Engine' object has no attribute 'cursor'


C:\Users\villy\AppData\Local\Temp\ipykernel_35792\567958940.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con=engine, schema=schema, if_exists='append', index=False)


In [7]:
import pandas as pd

from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from snowflake.sqlalchemy import URL

load_dotenv(override=True)

True

In [ ]:

def create_snowflake_engine():
    try:
        # Retrieve Snowflake connection parameters from environment variables
        snowflake_user = os.getenv('sn_user')
        snowflake_password = os.getenv('sn_password')
        snowflake_account = os.getenv('sn_account_identifier')
        snowflake_warehouse = os.getenv('sn_warehouse')
        snowflake_database = os.getenv('sn_database')
        snowflake_schema = os.getenv('sn_schema')
        snowflake_role = os.getenv('sn_role')

        # Create Snowflake URL
        snowflake_url = URL(
            user=snowflake_user,
            password=snowflake_password,
            account=snowflake_account,
            database=snowflake_database,
            schema=snowflake_schema,
            warehouse=snowflake_warehouse,
            role=snowflake_role
        )

        # Create SQLAlchemy engine and return it
        engine = create_engine(snowflake_url)
        return engine

    except Exception as e:
        print(f"Error creating Snowflake engine: {e}")
        return None

def load_csv_to_snowflake(engine, schema, table_name, csv_file_path):
    try:
        # Load CSV into Pandas DataFrame
        df = pd.read_csv(csv_file_path)

        # Write DataFrame to Snowflake table using SQLAlchemy Engine
        df.to_sql(table_name, con=engine, schema=schema, if_exists='append', index=False)

        print(f"Successfully loaded CSV data into Snowflake table '{schema}.{table_name}'")

    except Exception as e:
        print(f"Error loading CSV data to Snowflake: {e}")

if __name__ == "__main__":
    # Example usage
    snowflake_engine = create_snowflake_engine()

    if snowflake_engine:
        schema = 'STG'
        table_name = 'tripsdata'
        csv_file_path = './raw_data/tripsdata.csv'

        load_csv_to_snowflake(snowflake_engine, schema, table_name, csv_file_path)
    else:
        print("Failed to create Snowflake engine.")


In [4]:
import pandas as pd

from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from snowflake.sqlalchemy import URL

load_dotenv(override=True)

True

In [5]:


from db_utils import get_client, get_snowflake_engine
from extract_clickhouse import fetch_data

client = get_client()
engine = get_snowflake_engine()

# Execute the fetch data with parameters client and query
fetch_data(client=client, engine=engine)

DatabaseError: HTTPDriver for https://github.demo.trial.altinity.cloud:8443 returned response code 400)
 Code: 6. DB::Exception: Cannot parse string '2015-05-01 ' as Date: syntax error at position 10 (parsed just '2015-05-01'): In scope SELECT pickup_date, vendor_id, passenger_count, trip_distance, payment_type, fare_amount, tip_amount FROM tripdata WHERE pickup_date = (toDate('2015-05-01 ') + 1). (CANNOT_PARSE_TEXT) (version 24.5.3.5 (official build))
